## Generate plots for analysed data
The corresponding experiment data is stored as pickle files in the directory _figures_. Run the setup cells to get started.

# Setup

##### Set up libaries, paths and data

In [ ]:
%matplotlib inline
import sys, os
import math
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display
import time
import pprint
import glob
import netCDF4
import matplotlib.gridspec as gridspec
from matplotlib.patches import ConnectionPatch
sys.path.append('postdales')
import postdales as da

##### Define the plot style
Can be configured in os.path.join(mpl._get_configdir(), 'stylelib').
Default styles are in os.path.join(mpl.get_data_path(), 'stylelib').

In [ ]:
# define standard colors
light, dark, emphcolors = da.tools.colorschemes()
# define additional plot styles
linestyles = da.tools.plotstyles()
# define the plot style
mpl.style.use(dark)
# mpl.style.use(light)

### _Select the experiment numbers:_

In [ ]:
expnrs = ['521', '522', '593', '574', '545', '556', '500']

### Load saved exp data object

In [ ]:
# set up list of exp class object
exps = []
for expnr in expnrs:
    exp = da.tools.load_saved_exp(expnr, 'figures/')
    exps.append(exp)

# Contents

#### Compare block inputs
1. [Blocks](#Blocks)    
    a) [Geometries](#Blocks-geometries)    
    b) [Building fraction](#Blocks:-building-fraction)    
    c) [Frontal area](#Blocks:-frontal-area)    
    d) [Layouts and domain](#Blocks-layouts-and-domain)  
    e) [Layouts and blockfuntions](#Blocks-layouts-and-blockfunctions)  
2. [Canyons](#Canyons)
    
#### Compare output statistics
3. [Mean wind profiles](#Mean-wind-profiles)    
    a) [Wind profiles](#Mean-wind-profiles)    
    b) [Normalised wind profiles](#Normalised-wind-profiles)   
    c) [Normalised wind profiles in the urban canopy](#Normalised-wind-profiles-in-the-urban-canopy)   
4. [Mean velocity flow regimes](#Mean-velocity-flow-regimes)    
    a) [Homogeneous and heterogeneous heights](#Homogeneous-and-heterogeneous-heights)    
    b) [Representative homogeneous and heterogeneous heights](#Representative-homogeneous-and-heterogeneous-heights)    
5. [Comparing integrated forces](#Comparing-integrated-fluxes,-drag-and-stress)    
    a) [Total vertical momentum fluxes](#Total-vertical-momentum-fluxes)    
    b) [Integrated drag](#Integrated-drag)    
    c) [Total stress](#Total-stress)  
    
#### Analysis of vertical profiles
6. [Vertical wind and flux profiles](#Vertical-wind-and-flux-profiles)
7. [Normalised vertical profiles and estimations](#Normalised-vertical-profiles-and-estimations)    
8. [Wind profile and momentum flux developments](#Wind-profile-and-momentum-flux-developments)
9. [Force balance and integrated force balance](#Force-balance-and-integrated-force-balance)
10. [Force balance with ISL and RSL](#Force-balance-with-ISL-and-RSL)
11. [Stress profiles (CSA)](#Stress-profiles-(CSA))
12. [Force balance example](#Force-balance-example)
13. [Difference CSA and ISA](#Difference-CSA-and-ISA)

#### Parameter estimations
13. [Friction velocity estimates](#Friction-velocity-estimates)
14. [Blending height estimates](#Blending-height-estimates)
15. [Log law estimates](#Log-law-estimates)

#### Field plots
[Read in field data](#Read-in-field-data)
16. [Field plot](#Field-plot)
17. [Field plot with mini layout](#Field-plot-with-mini-layout)
18. [Quiver and wind speed plot](#Quiver-and-wind-speed-plot)
19. [Field plot with transformed field](#Field-plot-with-transformed-field)
20. [Compare field plots with equal limits](#Compare-field-plots-with-equal-limits)
21. [Video of field plots](#Video-of-field-plots)
22. [Flow field decomposition into time-space average, dispersive and turbulent-term](#Flow-field-decomposition-into-time-space-average,-dispersive-and-turbulent-term.)

---
# Compare block inputs

## Blocks

[Back to Contents](#Contents)
### Blocks geometries

In [ ]:
# setup
if len(exps) == 7:
    n = len(exps) - 1  # ignore reference solution
    col = 3
elif len(exps) == 1:
    n = 1
    col = 1
else:
    n = len(exps)
    col = 3
row = math.ceil(n/col)
i = 0

# plot
fig = plt.figure(figsize=(3*col, 3*row))
for exp in exps[:n]:
    # how often block layouts are duplicated
    divx = exp.xsize/120  # 4
    divy = exp.ysize/120  # 2
    nbl = int(len(exp.dimblocks)/(divx * divy))    
    limits=[0, exp.xsize/divx, 0, exp.ysize/divy, 0, exp.zsize/2]
    i += 1
    ax = fig.add_subplot(row, col, i, projection='3d')
#     fig = plt.figure(figsize=(3, 3))  # extra figure for each exp
#     ax = fig.add_subplot(1, 1, 1, projection='3d')  # extra figure for each exp
    da.plot.blocks3d(exp.dimblocks[:nbl], ax=ax, limits=limits, 
                       edgecolors='black', facecolor=exp.color)
    plt.title(exp.reference)
    ax.xaxis.set_ticks(np.linspace(*limits[0:2], 5))
    ax.yaxis.set_ticks(np.linspace(*limits[2:4], 5))
    ax.zaxis.set_ticks(np.linspace(*limits[4:6], 4))
    ax.set_aspect('equal')
    ax.view_init(25, 235)
    ax.dist=11
    # check whether dark or light plotstyle
    textcolor = ax.xaxis._axinfo['tick']['color']
    # plot wind direction
    plt.annotate('', xy=(0.45, 0.685), xycoords='axes fraction', 
                 xytext=(0.25, 0.63), textcoords='axes fraction', 
                 arrowprops=dict(facecolor=textcolor, arrowstyle="simple"))
    plt.annotate('wind', xy=(0.27, 0.685), xycoords='axes fraction', rotation=15)
plt.tight_layout()

[Back to Contents](#Contents)
### Blocks: building fraction

In [ ]:
# setup
if len(exps) == 7:
    n = len(exps) - 1  # ignore reference solution
    col = 3
elif len(exps) == 1:
    n = 1
    col = 1
else:
    n = len(exps)
    col = 3
row = math.ceil(n/col)
i = 0

# plot building density lambda_p
fig = plt.figure(figsize=(3*col, 3*row))
i = 0
for exp in exps[:n]:
    # how often block layouts are duplicated
    divx = exp.xsize/120  # 4
    divy = exp.ysize/120  # 2
    nbl = int(len(exp.dimblocks)/(divx * divy))
    limits=[0, exp.xsize/divx, 0, exp.ysize/divy, 0, exp.zsize/2]
    roofs = da.plot.roofs(exp.dimblocks[:nbl])
    # fix bug in matplotlib: make roofs slightly higher than buildings
    for roof in roofs:
        for r in roof:
            r[-1] += 0.01
    i += 1
    ax = fig.add_subplot(row, col, i, projection='3d')
#     fig = plt.figure(figsize=(3, 3))  # extra figure for each exp
#     ax = fig.add_subplot(1, 1, 1, projection='3d')  # extra figure for each exp
    da.plot.blocks3d(exp.dimblocks[:nbl], ax=ax, limits=limits, edgecolors='black', facecolor='xkcd:light grey')
    da.plot.plane3d(roofs, ax=ax, facecolor='tab:orange')
    plt.title(exp.reference)
    ax.xaxis.set_ticks(np.linspace(*limits[0:2], 5))
    ax.yaxis.set_ticks(np.linspace(*limits[2:4], 5))
    ax.zaxis.set_ticks(np.linspace(*limits[4:6], 4))
    ax.w_zaxis.set_pane_color(mpl.colors.to_rgb('xkcd:light grey'))
    ax.set_aspect('equal')
    ax.view_init(25, 235)
    ax.dist=11
    # plot wind direction
    plt.annotate('', xy=(0.45, 0.685), xycoords='axes fraction', 
                 xytext=(0.25, 0.63), textcoords='axes fraction', 
                 arrowprops=dict(facecolor='w', arrowstyle="simple"))
    plt.annotate('wind', xy=(0.27, 0.685), xycoords='axes fraction', rotation=15)
plt.suptitle(r"Building area density $\lambda_p$")
plt.tight_layout()
plt.subplots_adjust(top=0.92)
plt.show()

[Back to Contents](#Contents)
### Blocks: frontal area

In [ ]:
# setup
if len(exps) == 7:
    n = len(exps) - 1  # ignore reference solution
    col = 3
elif len(exps) == 1:
    n = 1
    col = 1
else:
    n = len(exps)
    col = 3
row = math.ceil(n/col)
i = 0
# how often block layouts are duplicated
divx = exp.xsize/120  # 4
divy = exp.ysize/120  # 2

# plot building density lambda_p
fig = plt.figure(figsize=(3*col, 3*row))
for exp in exps[:n]:
    nbl = int(len(exp.dimblocks)/(divx * divy))
    limits=[0, exp.xsize/divx, 0, exp.ysize/divy, 0, exp.zsize/2]
    fronts = da.plot.fronts(exp.dimblocks[:nbl])
    # fix bug in matplotlib: make fronts slightly more left than buildings
    for front in fronts:
        for f in front:
            f[0] -= 0.01
    i += 1
    ax = fig.add_subplot(row, col, i, projection='3d')
#     fig = plt.figure(figsize=(3, 3))  # extra figure for each exp
#     ax = fig.add_subplot(1, 1, 1, projection='3d')  # extra figure for each exp
    da.plot.blocks3d(exp.dimblocks[:nbl], ax=ax, limits=limits, edgecolors='black', facecolor='xkcd:light grey')
    da.plot.plane3d(fronts, ax=ax, facecolor='tab:blue')
    plt.title(exp.reference)
    ax.xaxis.set_ticks(np.linspace(*limits[0:2], 5))
    ax.yaxis.set_ticks(np.linspace(*limits[2:4], 5))
    ax.zaxis.set_ticks(np.linspace(*limits[4:6], 4))
    ax.w_zaxis.set_pane_color(mpl.colors.to_rgb('xkcd:light grey'))
    ax.set_aspect('equal')
    ax.view_init(25, 235)
    ax.dist=11
    # plot wind direction
    plt.annotate('', xy=(0.45, 0.685), xycoords='axes fraction', 
                 xytext=(0.25, 0.63), textcoords='axes fraction', 
                 arrowprops=dict(facecolor='w', arrowstyle="simple"))
    plt.annotate('wind', xy=(0.27, 0.685), xycoords='axes fraction', rotation=15)
plt.suptitle(r"Building area density $\lambda_p$")
plt.tight_layout()
plt.subplots_adjust(top=0.92)
plt.show()

[Back to Contents](#Contents)
### Blocks layouts and domain

In [ ]:
# layouts = exps[0:3] + exps[4:6]
layouts = exps[0:6]
example = exps[5]

# setup
n = len(layouts)
col = n - 2
row = math.ceil(n/col)

# layout plots
i = 0
j = 0
fig = plt.figure(figsize=(4*(col-1), 3*row))
for exp in layouts:
    # how often block layouts are duplicated
    divx = int(exp.xsize/120)  # 4
    divy = int(exp.ysize/120)  # 2
    ax = plt.subplot2grid((row, col), (i, j%col))
    nbl = int(len(exp.dimblocks)/(divx * divy))
    limits = [0, exp.xsize/divx, 0, exp.ysize/divy]
    da.plot.layout(exp.dimblocks[:nbl], ax=ax, limits=limits, facecolor=exp.color)
    ax.set_xticks(np.linspace(*limits[0:2], 5))
    ax.set_yticks(np.linspace(*limits[2:4], 5))
#     if exp.reference == "(c)":
#         plt.title("(c), (d)")
#     else:
#         plt.title(exp.reference)
    plt.title(exp.reference)
    j += 1
    if j%col == 0:
        i += 1
    ax.dist=11   
    
# domain plot   
ax2 = plt.subplot2grid((row, col), (i, j%col), colspan=col-1)
dlimits = [0, example.xsize, 0, example.ysize]
da.plot.layout(example.dimblocks, ax=ax2, limits=dlimits, facecolor='grey')
xticks = [(i/divx)*example.xsize for i in range(divx + 1)]
yticks = [(i/divy)*example.ysize for i in range(divy + 1)]
ax2.set_yticks(yticks)
ax2.set_xticks(xticks)

# frame sample layout
da.plot.layout([[0, example.xsize/divx, 0, example.ysize/divy, 0, 0]], limits=[0, example.xsize, 0, example.ysize], facecolor=(0, 0, 0, 0), edgecolor='k')
plt.title("Domain")
# connection lines
con1 = ConnectionPatch(xyA=(example.xsize/divx, 0), xyB=(0, 0), coordsA="data", coordsB="data",
                       axesA=ax, axesB=ax2, 
                       linestyle=':', linewidth=0.6, color='k')
ax.add_artist(con1)
con2 = ConnectionPatch(xyA=(example.xsize/divx, example.ysize/divy), xyB=(0, example.ysize/divy), coordsA="data", coordsB="data",
                       axesA=ax, axesB=ax2, 
                       linestyle=':', linewidth=0.6, color='k')
ax.add_artist(con2)
# dotted seperator lines
for i in range (divx):
    con3 = ConnectionPatch(xyA=(((1+i)/divx)*example.xsize, 0), xyB=(((1+i)/divx)*example.xsize, example.ysize), coordsA="data", coordsB="data", axesA=ax2, axesB=ax2, 
                           linestyle=':', linewidth=0.6, color='k')
    ax2.add_artist(con3)
for i in range (divy):
    con4 = ConnectionPatch(xyA=(0, ((1+i)/divy)*example.ysize), xyB=(example.xsize, ((1+i)/divy)*example.ysize), coordsA="data", coordsB="data", axesA=ax2, axesB=ax2, 
                           linestyle=':', linewidth=0.6, color='k')
    ax2.add_artist(con4)
    
plt.tight_layout()
plt.show()

[Back to Contents](#Contents)
### Blocks layouts and blockfunctions

#### with $ \lambda_p $

In [ ]:
for exp in exps:
    # how often block layouts are duplicated
    divx = exp.xsize/120  # 4
    divy = exp.ysize/120  # 2
    nbl = int(len(exp.dimblocks)/(divx * divy))    
    limits=[0, exp.xsize/divx, 0, exp.ysize/divy, 0, exp.zsize/2]
    # plot
    fig = plt.figure(figsize=(9, 4))
    # geometry plot
    ax = fig.add_subplot(1, 3, 1, projection='3d')
    da.plot.blocks3d(exp.dimblocks[:nbl], ax=ax, 
                       facecolor=exp.color, edgecolor='k', 
                       limits=limits)
    plt.title("Blocks")
    ax.xaxis.set_ticks(np.linspace(*limits[0:2], 5))
    ax.yaxis.set_ticks(np.linspace(*limits[2:4], 5))
    ax.zaxis.set_ticks(np.linspace(*limits[4:6], 4))
    ax.set_aspect('equal')
    ax.view_init(25, 235)
#     ax.dist=9
    # layout plot
    ax = fig.add_subplot(1, 3, 2)
    da.plot.layout(exp.dimblocks[:nbl], ax=ax, limits=limits[:4], 
                     facecolor=exp.color)
    ax.set_xticks(np.linspace(*limits[0:2], 5))
    ax.set_yticks(np.linspace(*limits[2:4], 5))
    plt.title("Blocks plan layout")    
    # building density plot
    ax = fig.add_subplot(1, 3, 3)
    plt.plot(exp.blockfunction_lp, exp.zm*exp.normh, color=exp.color)
    plt.axhline(y=exp.zh*exp.normh, linestyle=":", color='grey')
    plt.xlabel("$A_B/A_T$")
    plt.ylabel("$z/z_H$")
    plt.title("Building area density")
    plt.suptitle("%s" %exp.reference)
    plt.tight_layout()
    plt.subplots_adjust(top=0.8)
    plt.show()

#### with $\lambda_p $ and $\lambda_f$ (continuous)

In [ ]:
# setup
if len(exps) == 7:
    n = len(exps) - 1  # ignore reference solution
else:
    n = len(exps)
for exp in exps[:n]:
    # how often block layouts are duplicated
    divx = exp.xsize/120  # 4
    divy = exp.ysize/120  # 2
    nbl = int(len(exp.dimblocks)/(divx * divy))    
    limits=[0, exp.xsize/divx, 0, exp.ysize/divy, 0, exp.zsize/2]
    # plot
    fig = plt.figure(figsize=(12, 4))
    # geometry plot
    ax = fig.add_subplot(1, 4, 1, projection='3d')
    da.plot.blocks3d(exp.dimblocks[:nbl], ax=ax, 
                       facecolor=exp.color, edgecolor='k', 
                       limits=limits)
    plt.title("Blocks")
    ax.xaxis.set_ticks(np.linspace(*limits[0:2], 5))
    ax.yaxis.set_ticks(np.linspace(*limits[2:4], 5))
    ax.zaxis.set_ticks(np.linspace(*limits[4:6], 4))
    ax.set_aspect('equal')
    ax.view_init(25, 235)
#     ax.dist=9
    # layout plot
    ax = fig.add_subplot(1, 4, 2)
    da.plot.layout(exp.dimblocks[:nbl], ax=ax, limits=limits[:4], 
                     facecolor=exp.color)
    ax.set_xticks(np.linspace(*limits[0:2], 5))
    ax.set_yticks(np.linspace(*limits[2:4], 5))
    plt.title("Blocks plan layout")    
    # building density plot
    ax = fig.add_subplot(1, 4, 3)
    plt.plot(exp.blockfunction_lp, exp.zm*exp.normh, color=exp.color)
    plt.axhline(y=1, linestyle=":", color='grey')
    plt.xlabel("$\lambda_p$")
    plt.ylabel("$z/z_H$")
    plt.title("Building area density")
    # building frontal density plot
    ax = fig.add_subplot(1, 4, 4)
    plt.plot(exp.blockfunction_lf, exp.zm*exp.normh, color=exp.color)
    plt.axhline(y=1, linestyle=":", color='grey')
    plt.xlabel("$\lambda_f$")
    plt.ylabel("$z/z_H$")
    plt.title("Building frontal area density")

    plt.suptitle("%s" %exp.reference)
    plt.tight_layout()
    plt.subplots_adjust(top=0.8)
    plt.show()

#### with $\lambda_p$ and $\lambda_f$ (continuous and discrete)

In [ ]:
# setup
if len(exps) == 7:
    n = len(exps) - 1  # ignore reference solution
else:
    n = len(exps)
for exp in exps[:n]:
    # how often block layouts are duplicated
    divx = exp.xsize/120  # 4
    divy = exp.ysize/120  # 2
    nbl = int(len(exp.dimblocks)/(divx * divy))    
    limits=[0, exp.xsize/divx, 0, exp.ysize/divy, 0, exp.zsize/3]
    # plot
    fig = plt.figure(figsize=(15, 4))
    # geometry plot
    ax = fig.add_subplot(1, 5, 1, projection='3d')
    da.plot.blocks3d(exp.dimblocks[:nbl], ax=ax, 
                       facecolor=exp.color, edgecolor='k', 
                       limits=limits)
    plt.title("Blocks")
    ax.xaxis.set_ticks(np.linspace(*limits[0:2], 5))
    ax.yaxis.set_ticks(np.linspace(*limits[2:4], 5))
    ax.zaxis.set_ticks(np.linspace(*limits[4:6], 4))
    ax.set_aspect('equal')
    ax.view_init(25, 235)
    ax.dist=12
    # layout plot
    ax = fig.add_subplot(1, 5, 2)
    da.plot.layout(exp.dimblocks[:nbl], ax=ax, limits=limits[:4], 
                     facecolor=exp.color)
    ax.set_xticks(np.linspace(*limits[0:2], 5))
    ax.set_yticks(np.linspace(*limits[2:4], 5))
    plt.title("Blocks plan layout")    
    # building density plot
    ax = fig.add_subplot(1, 5, 3)
    plt.plot(exp.blockfunction_lp, exp.zm*exp.normh, color=exp.color)
    plt.axhline(y=exp.zh*exp.normh, linestyle=":", color='grey')
    plt.xlabel("$\lambda_p$")
    plt.ylabel("$z/z_H$")
    plt.title("Building area density")
    # building frontal density plot
    ax = fig.add_subplot(1, 5, 4)
    plt.plot(exp.blockfunction_lf, exp.zm*exp.normh, color=exp.color)
    plt.axhline(y=exp.zh*exp.normh, linestyle=":", color='grey')
    plt.xlabel("$\lambda_f$")
    plt.ylabel("$z/z_H$")
    plt.title("Building frontal area density")
    # building frontal density plot
    ax = fig.add_subplot(1, 5, 5)
    plt.plot(exp.blockfunction_lf_alt, exp.zm*exp.normh, color=exp.color)
    plt.axhline(y=exp.zh*exp.normh, linestyle=":", color='grey')
    plt.xlabel("$\lambda_f$")
    plt.ylabel("$z/z_H$")
    plt.title("Building frontal area density")

    plt.suptitle("%s" %exp.reference)
    plt.tight_layout()
    plt.subplots_adjust(top=0.8)
    plt.show()

[Back to Contents](#Contents)
## Canyons

In [ ]:
for exp in exps:
    fig = plt.figure(figsize=(12, 3))
    ax = fig.add_subplot(1, 3, 1)
    colors = sns.color_palette("cubehelix", len(exp.fullcanyons))
    for canyon, color in zip(exp.fullcanyons, colors):
        da.plot.layout([canyon], ax=ax, limits=[0, exp.imax, 0, exp.jtot], facecolor=color)
    plt.title("Street canyons")
    ax = fig.add_subplot(1, 3, 2)
    da.plot.layout(exp.blocks, ax=ax, limits=[0, exp.imax, 0, exp.jtot], facecolor=exp.color)
    da.plot.layout(exp.canyons, ax=ax, limits=[0, exp.imax, 0, exp.jtot], facecolor="xkcd:light grey")
    plt.title("original layout")
    ax = fig.add_subplot(1, 3, 3)
    da.plot.layout(exp.dimblocks, ax=ax, limits=[0, exp.xsize, 0, exp.ysize], facecolor=exp.color)
    colors = sns.color_palette("cubehelix", len(exp.dimcanyons))
    for canyon, color in zip(exp.dimcanyons, colors):
        da.plot.layout([canyon], ax=ax, limits=[0, exp.xsize, 0, exp.ysize], facecolor=color)
    plt.title("dimensionalised")
    plt.show()

[Back to Contents](#Contents)

---
# Compare output statistics

## Mean wind profiles

In [ ]:
# show mean wind profiles
i=0
plt.figure()
for exp in exps:
    plt.plot(exp.uav, exp.zt, label=exp.reference, marker=exp.marker, markevery=100+i, color=exp.color)
    i +=10
plt.legend()
plt.xlabel(r"Wind speed $\langle \bar u \rangle(z)$ [m/s]")
plt.ylabel('Height z [m]')
plt.title("Mean wind profiles")
plt.show()
for exp in exps:
    dzs = np.gradient(exp.zt)
    print(exp.reference, "height average:", np.sum(exp.uav*dzs)/exp.zsize)

[Back to Contents](#Contents)
### Normalised wind profiles

In [ ]:
i = 0
plt.figure()
for exp in exps:
    uref = exp.uav[-1]
#     uref = exp.ustar_isa
#     hnorm = exp.zh
#     hnorm = exp.zmax
    hnorm = 1
    plt.plot(exp.uav/uref, exp.zt/hnorm, label=exp.reference, color=exp.color, marker=exp.marker, markevery=100 + i)
    i += 10
plt.legend()
plt.xlabel(r"Normalised wind speed $\langle \bar u \rangle/u_{ref}$")
plt.ylabel('Height z [m]')
plt.title("Normalised mean wind profiles")
plt.show()

[Back to Contents](#Contents)
### Normalised wind profiles in the urban canopy

In [ ]:
i = 0
markevery = [4, 2, 2, 4, 3, 3]
plt.figure()
for exp in exps:
    zref = exp.zmax_tindex
#     uref = exp.ustar
#     uref = 2
#     uref = exp.uav[-1]
    uref = exp.uav[zref]
    plt.plot(exp.uav/uref, exp.zt/exp.zt[zref], label=exp.reference, color=exp.color, 
            marker=exp.marker, markevery=markevery[i])
    i += 1
plt.axhline(y=1, color='grey', linewidth=0.5)
plt.legend()
plt.ylim(0, 2.5) # toggle on and off to see full profile
# plt.xlim(right=10)
plt.xlabel(r"Normalised mean wind speed $\langle \bar u \rangle/u_{\ast}$")
plt.ylabel('Height $z/z_{\max}$')
plt.title("Normalised wind profiles in the urban canopy")
plt.show()

[Back to Contents](#Contents)
## Mean velocity flow regimes

### Homogeneous and heterogeneous heights

In [ ]:
fig = plt.figure(figsize=(6, 4))
ax1 = fig.add_subplot(1, 2, 1)
plt.xlabel(r"Wind speed $\langle \bar u \rangle(z)$ [m/s]")
plt.ylabel('Height z [m]')
plt.title("Homogeneous heights")
ax2 = fig.add_subplot(1, 2, 2)
plt.xlabel(r"Wind speed $\langle \bar u \rangle(z)$ [m/s]")
plt.ylabel('Height z [m]')
plt.title("Heterogeneous heights")
i = 0
for exp in exps:   
    if exp.reference in ['(a)', '(b)', '(c)']:
        ax1.plot(exp.uav, exp.zt, label=exp.reference, color=exp.color, 
                 marker=exp.marker, markevery=100 + i)
    elif exp.reference in ['(d)', '(e)', '(f)']:        
        ax2.plot(exp.uav, exp.zt, label=exp.reference, color=exp.color, 
                 marker=exp.marker, markevery=100 + i)
    i += 15
ax1.legend()
ax2.legend()
plt.tight_layout()
plt.show()

[Back to Contents](#Contents)
### Representative homogeneous and heterogeneous heights

In [ ]:
fig = plt.figure(figsize=(3*3, 4))
ax1 = fig.add_subplot(1, 3, 1)
ax1.set_title("Flat surface")
ax2 = fig.add_subplot(1, 3, 2)
ax2.set_title("Homogeneous heights")
ax3 = fig.add_subplot(1, 3, 3)
ax3.set_title("Heterogeneous heights")
# set labels
for ax in [ax1, ax2, ax3]:
    ax.set_xlabel(r"Wind speed $\langle \bar u \rangle(z)$ [m/s]")
    ax.set_ylabel('Height z [m]')
    ax.set_ylim(top=exp.zsize)
for exp in exps:        
    if exp.reference in ['(g)']:
        ax1.plot(exp.uav, exp.zt, label=exp.reference, color=exp.color, marker=exp.marker, markevery=[0, 145])
        # log law
        ax1.plot(exp.uav[:-30], exp.loglaw[:-30], label="log law", color='xkcd:bluish grey', linestyle="--", linewidth=0.6)
        ax1.text(exp.uav[0], (exp.loglaw[0]+5), "log law", style='italic', color='grey', fontsize=8)  
    if exp.reference in ['(b)']:
        ax2.plot(exp.uav, exp.zt, label=exp.reference, color=exp.color, marker=exp.marker, markevery=[0, 145])
#         # z_H
#         ax2.scatter(exp.uav[exp.zh_tindex], exp.zh, marker='.', color='grey')
#         ax2.text(exp.uav[exp.zh_tindex]-0.1, (exp.zh-8), "$z_H$", fontsize=8, color='grey')
        # log law
        ax2.plot(exp.uav, exp.loglaw, label="log law", color='xkcd:bluish grey', linestyle="--", linewidth=0.6)
        ax2.text(exp.uav[1]-0.05, (exp.loglaw[0]+5), "log law", style='italic', color='grey', fontsize=8)  
#         # z_e
#         plt.scatter(exp.uav[exp.ze_index], exp.ze, marker='.', color='k')
#         plt.text(exp.uav[exp.ze_index]-0.1, (exp.ze-8), "$z_e$", fontsize=8, color='k')
    elif exp.reference in ['(d)']:
        ax3.plot(exp.uav, exp.zt, label=exp.reference, color=exp.color, marker=exp.marker, markevery=[0, 145])
#         # z_H
#         ax3.scatter(exp.uav[exp.zh_tindex], exp.zh, marker='.', color='grey')
#         ax3.text(exp.uav[exp.zh_tindex]-0.1, (exp.zh-8), "$z_H$", fontsize=8, color='grey')
#         # z_max
#         ax3.scatter(exp.uav[exp.zmax_tindex], exp.zmax, marker='.', color='grey')
#         plt.text(exp.uav[exp.zmax_tindex]-0.15, (exp.zmax+5), "$z_{max}$", fontsize=8, color='grey')
        # log law
        ax3.plot(exp.uav, exp.loglaw, label="log law", color='xkcd:bluish grey', linestyle="--", linewidth=0.6)
        ax3.text(exp.uav[1]-0.05, (exp.loglaw[0]+5), "log law", style='italic', color='grey', fontsize=8)  
#         # z_e
#         plt.scatter(exp.uav[exp.ze_index], exp.ze, marker='.', color='k')
#         plt.text(exp.uav[exp.ze_index]-0.1, (exp.ze-8), "$z_e$", fontsize=8, color='k')
plt.tight_layout()
plt.show()

[Back to Contents](#Contents)
## Comparing integrated fluxes, drag and stress

### Total vertical momentum fluxes

In [ ]:
#total fluxes CSA
plt.figure()
i = 0
for exp in exps:
    i += 5
    plt.plot(exp.totflux, exp.zm, label=exp.reference, color=exp.color, marker=exp.marker, markevery=[2, 100 + i])
    plt.plot(exp.totstress[:exp.zmax_index+3], exp.zm[:exp.zmax_index+3], 
             color=exp.color, linestyle="--", linewidth=0.5,
             marker=exp.marker, markevery=[0])
plt.legend()
plt.title("Total vertical momentum flux")
plt.xlabel(r"Flux $\tau(z)$ [m$^2$/s$^2$]")
plt.ylabel('Height z [m]')
plt.show()

#### Normalised

In [ ]:
#total fluxes CSA
plt.figure()
i = 0
for exp in exps:
    try:
        normu = exp.normu
    except Exception:
        normu = np.array(1)
    i += 5
    plt.plot(exp.totflux*normu, exp.zm, label=exp.reference, color=exp.color, marker=exp.marker, markevery=[1, 100 + i])
    plt.plot(exp.totstress[:exp.zmax_index+3]*normu, exp.zm[:exp.zmax_index+3], 
             color=exp.color, linestyle="--", linewidth=0.5,
             marker=exp.marker, markevery=[1])
plt.legend()
plt.title("Total vertical momentum fluxes")
plt.xlabel(r"Normalised flux $\tau(z)/u_{\ast g}^2$")
# plt.xlabel(r"Flux $\tau(z)$ [m$^2$/s$^2$]")
plt.ylabel('Height z [m]')
plt.show()

[Back to Contents](#Contents)
### Integrated drag

In [ ]:
# drag CSA
plt.figure()
i = 0
for exp in exps:
    i += 5
    plt.plot(exp.drag, exp.zm, label=exp.reference, color=exp.color, marker=exp.marker, markevery=[20+i])
    plt.plot(exp.totstress[2:], exp.zm[2:], color=exp.color, linestyle="--", linewidth=0.5, marker=exp.marker, markevery=[0])
plt.legend()
plt.title("Drag")
plt.xlabel("Integrated drag $\hat D(z)$ [m$^2$/s$^2$]")
plt.ylabel('Height z [m]')
plt.show()

[Back to Contents](#Contents)
### Total stress

In [ ]:
# stress
plt.figure()
i = 0
for exp in exps:
    i += 5
    plt.plot(exp.totstress, exp.zm, label=exp.reference, color=exp.color, marker=exp.marker, markevery=[0, 100 + i])
plt.legend()
plt.xlabel(r"Total stress $\hat F(z)$ [m$^2$/s$^2$]")
plt.ylabel('Height z [m]')
plt.show()

[Back to Contents](#Contents)

---
# Analysis of vertical profiles

## Vertical wind and flux profiles

In [ ]:
for exp in exps:
    # rounding time steps to nearest 10
    timestep = int(round(exp.t[-1]/10)*10)
    if len(exp.t) > 1:
        avtime = int(round((exp.t[-1] - exp.t[-2]) * 0.1) * 10)
        
    # plot figure
    fig = plt.figure(figsize=(3*2, 4*1))
    
    # mean wind
    ax = fig.add_subplot(1, 2, 1)
    plt.plot(exp.uav, exp.zt*exp.normh, color=exp.color)
    # z_H
    plt.scatter(exp.uav[exp.zh_tindex], exp.zh*exp.normh, marker='.', color='grey')
    plt.text(exp.uav[exp.zh_tindex]-0.08, (exp.zh-8)*exp.normh, "$z_H$", 
             fontsize=8, color='grey')
    # z_max
    plt.scatter(exp.uav[exp.zmax_tindex], exp.zmax*exp.normh, marker='.', color='grey')
    plt.text(exp.uav[exp.zmax_tindex]-0.22, (exp.zmax+5)*exp.normh, "$z_{max}$", 
             fontsize=8, color='grey')
    plt.xlabel(r"Wind speed $\langle \bar u \rangle(z)$ [m/s]")
    plt.ylabel('Height $z/z_H$')
    plt.title("Mean $u$ wind profile")

    # fluxes
    ax = fig.add_subplot(1, 2, 2)
    plt.plot(exp.totflux, exp.zm*exp.normh, color=exp.color)
    plt.plot(exp.turbflux, exp.zm*exp.normh, label="turbulent flux", 
             linestyle=":", linewidth=1, color=exp.color)
    plt.plot(exp.dispflux, exp.zm*exp.normh, label='dispersive flux', 
             linestyle="--", linewidth=1, color=exp.color)
    plt.plot(exp.viscflux, exp.zm*exp.normh, label='viscous SGS flux', 
             linestyle="-.", linewidth=0.6, color=exp.color)
    # z_H
    plt.axhline(y=exp.zh*exp.normh, linewidth=0.5,  color="grey")
    plt.scatter((-0.05)*(1/exp.normf), exp.zh*exp.normh, marker='.', color='grey')
    plt.text((-0.08)*(1/exp.normf), (exp.zh-8)*exp.normh, "$z_H$", 
             fontsize=8, color='grey')
    # z_max
    plt.axhline(y=exp.zmax*exp.normh, linewidth=0.5, color='grey')
    plt.scatter((-0.05)*(1/exp.normf), exp.zmax*exp.normh, marker='.', color='grey')
    plt.text((-0.08)*(1/exp.normf), (exp.zmax+5)*exp.normh, "$z_{max}$", 
             fontsize=8, color='grey')
    plt.xlabel(r"Flux $\tau(z)$ [m$^2$/s$^2$]")
    plt.ylabel(r'Height $z/z_H$')
    plt.title("Vertical $u$ momentum flux")
    plt.legend(prop={'size': 'small'})

    if len(exp.t) > 1:
        plt.suptitle("%s Mean profiles after %s s with average time %s s" 
                     % (exp.reference, timestep, avtime) )
    else:
        plt.suptitle("%s Mean profiles after %s s" % (exp.reference, timestep) )
    plt.tight_layout()
    plt.subplots_adjust(top=0.88)
    plt.show()

[Back to Contents](#Contents)
## Normalised vertical profiles and estimations

In [ ]:
for exp in exps:
    ufreestream = exp.uav[-1]
    # plot figure
    fig = plt.figure(figsize=(3*2, 4*1))
    # mean wind
    ax = fig.add_subplot(1, 2, 1)
    # get default text color black or white
    textcolor = ax.xaxis.label.get_color()
    plt.plot(exp.uav/ufreestream, exp.zt*exp.normh, color=exp.color)
    # z_H
    plt.scatter(exp.uav[exp.zh_tindex]/ufreestream, exp.zh*exp.normh, 
                marker='.', color='grey')
    plt.text(exp.uav[exp.zh_tindex]/ufreestream-0.05, (exp.zh-8)*exp.normh, 
             "$z_H$", fontsize=8, color='grey')
    # z_max
    plt.scatter(exp.uav[exp.zmax_tindex]/ufreestream, exp.zmax*exp.normh, 
                marker='.', color='grey')
    plt.text(exp.uav[exp.zmax_tindex]/ufreestream-0.05, (exp.zmax+5)*exp.normh, 
             "$z_{max}$", fontsize=8, color='grey')
    # z_e
    plt.scatter(exp.uav[exp.ze_index]/ufreestream, exp.ze*exp.normh, 
                marker='.', color=textcolor)
    plt.text(exp.uav[exp.ze_index]/ufreestream, (exp.ze-8)*exp.normh, 
             "$z_e$", fontsize=8)
    # log law
    plt.plot(exp.uav/ufreestream, exp.loglaw*exp.normh, 
             color='xkcd:bluish grey', linestyle="--", linewidth=0.6)
    plt.text(exp.uav[1]/ufreestream, (exp.loglaw[1]-8)*exp.normh, 
             "log law", style='italic', color='grey', fontsize=8)  
    # z_0 + z_d
    plt.scatter(exp.uav[1]/ufreestream, exp.loglaw[1]*exp.normh, 
                marker='.', color=textcolor)
    plt.text(exp.uav[1]/ufreestream, (exp.loglaw[1]+5)*exp.normh, 
             "$z_0 + z_d$", fontsize=8)       
    plt.xlabel(r"Wind speed $\langle \bar u \rangle/u_{\infty}$")
    plt.ylabel('Height $z/z_H$')
    plt.title("Normalised wind velocity")

    # fluxes
    ax = fig.add_subplot(1, 2, 2)
    # estimated stress
    plt.plot(exp.totstress*exp.normf, exp.zm*exp.normh, 
             color=exp.color, linestyle=":")
    plt.text(1, exp.zm[int(exp.zmax_index/2)]*exp.normh,
                 r"$\hat F$", color=exp.color, fontsize=8)  
    # estimated drag
    plt.plot(exp.drag*exp.normf, exp.zm*exp.normh, linestyle="-.", 
             color=exp.color)
    plt.text(0.25, exp.zm[int(exp.zh_index + 5)]*exp.normh, 
             r"$\hat D$", color=exp.color, fontsize=8)
    # totalflux
    plt.plot(exp.totflux*exp.normf, exp.zm*exp.normh, color=exp.color)
    plt.text(0.75, exp.zm[-1]*exp.normh/3, 
             r"$\tau$", color=exp.color, fontsize=8)
    # turbulent flux
    plt.plot(exp.turbflux*exp.normf, exp.zm*exp.normh, 
             label=r"$\langle \overline{w'u'} \rangle$", 
             linewidth=1, linestyle=":", color='xkcd:light brown')
    # dispersive flux
    plt.plot(exp.dispflux*exp.normf, exp.zm*exp.normh, 
             label=r"$\langle \bar{w}''\bar{u}'' \rangle$", 
             linewidth=1, linestyle="--", color='xkcd:greyish purple')
    # SGS flux
    plt.plot(exp.viscflux*exp.normf, exp.zm*exp.normh, 
             label=r'$\nu_{SGS} \frac{\partial \langle \bar{u} \rangle}{\partial z}$', 
             linestyle="-.", linewidth=0.6, color='xkcd:grey green')
    # z_max
    plt.axhline(y=exp.zmax*exp.normh, linewidth=0.5, color='grey')
    plt.scatter(0, exp.zmax*exp.normh, marker='.', color='grey')
    plt.text(0-0.08*np.max(exp.totflux), (exp.zmax+5)*exp.normh, r"$z_{\max}$", 
             fontsize=8, color='grey')
    # z_h
    plt.axhline(y=exp.zh*exp.normh, linewidth=0.5,  color="grey")
    plt.scatter(0, exp.zh*exp.normh, marker='.', color='grey')
    plt.text(0-0.08*np.max(exp.totflux), (exp.zh-8)*exp.normh, "$z_H$", 
             fontsize=8, color='grey')
    
    # centre of mass Dm
    # only works because of vertical resolution delta z = 1
    plt.scatter(exp.drag[int(exp.drag_masscentre_csa)]*exp.normf, 
                exp.drag_masscentre_csa*exp.normh, 
                marker='.', color=textcolor)
    plt.text(exp.drag[int(exp.drag_masscentre_csa)]*exp.normf, 
             (exp.drag_masscentre_csa-8)*exp.normh, 
                 "$D_m$", fontsize=8)    

    plt.title("Normalised stresses")
    plt.ylabel(r'Height $z/z_H$')
    plt.xlabel(r"Fluxes, drag, stress $Q/u_{\ast}$")
    plt.legend(fontsize='small', labelspacing=0.5)
    
    plt.suptitle("%s" % exp.reference)
    plt.tight_layout()
    plt.subplots_adjust(top=0.88)
    plt.show()

[Back to Contents](#Contents)
## Wind profile and momentum flux developments

In [ ]:
# setup
varstart = -4
varstop = None
trange = slice(varstart, varstop)
# plot
for exp in exps:    
    try:
        times = exp.t[trange]
        varav = exp.uavs[trange]
        tots = exp.totfluxes[trange]
        turbs = exp.turbfluxes[trange]
        disps = exp.dispfluxes[trange]
    except Exception: # if slice out of bounds
        times = exp.t[:]
        varav = exp.uavs[:]  
        tots = exp.totfluxes[:]
        turbs = exp.turbfluxes[:]
        disps = exp.dispfluxes[:]  
        
    fig = plt.figure(figsize=(3*2, 4*1), dpi=250)
    # wind profile
    ax1 = fig.add_subplot(1, 2, 1)

    # get default text color black or white
    textcol = plt.xlabel('').get_color()
    if textcol is 'k':  # for dark background
        reverse=False
    else:
        reverse=True
    colorpal = sns.light_palette(exp.color, len(varav), reverse=reverse)

    for i, uav in enumerate(varav):
        # define variable length and average time
        if i - 1 < 0:
            try:
                t0 = int(exp.t[varstart - 1])
            except Exception:
                t0 = 0
        else:
            t0 = int(times[i-1])
        tstep = int(times[i])
        plt.plot(uav, exp.zt*exp.normh, label="%s s to %s s" %(t0, tstep),
                color=colorpal[i])
    plt.legend(fontsize='small', labelspacing=0.5)    
    plt.title("Wind profile development")
    plt.xlabel(r"Wind speed $\langle \bar u \rangle(z)$ [m/s]")
    plt.ylabel('Height $z/z_H$')

    # fluxes
    ax2 = fig.add_subplot(1, 2, 2)
    i = 0
    for tot, disp, turb in zip(tots, disps, turbs):
        # define variable length and average time
        if i - 1 < 0:
            try:
                t0 = int(exp.t[varstart - 1])
            except Exception:
                t0 = 0
        else:
            t0 = int(times[i-1])
        tstep = int(times[i])
        p = plt.plot(tot, exp.zm*exp.normh, label="%s s to %s s" %(t0, tstep),
                    color=colorpal[i])
        color = p[0].get_color()
        plt.plot(turb, exp.zm*exp.normh, linestyle=":", color=color)
        plt.plot(disp, exp.zm*exp.normh, linestyle="--", color=color)
        i += 1

#     plt.legend()
    plt.title("Flux development %s" % exp.name)
    plt.xlabel(r"Flux $\tau(z)$ [m$^2$/s$^2$]")
    plt.ylabel('Height $z/z_H$')
    
    plt.tight_layout()
    plt.show()

[Back to Contents](#Contents)

## Force balance and integrated force balance

In [ ]:
for exp in exps:
    # define fluxes (old data doesn't have the gradients)
    dtau = exp.totflux_gradient
    dturb =  exp.turbflux_gradient
    ddisp =  exp.dispflux_gradient
    dvisc =  exp.viscflux_gradient
    
    # plot figure
    fig = plt.figure(figsize=(3*2, 4*1), dpi=250)
    # force balance
    ax1 = fig.add_subplot(1, 2, 1)
    # forcing    
    plt.plot(exp.forcing, exp.zm, linestyle=linestyles[0], color=exp.color)
#             , label=r"$F$")
    # drag
    plt.plot(exp.dragforce, exp.zm, linestyle=linestyles[11], color=exp.color) 
#              , label=r"$D$")
    # total flux
    plt.plot(dtau, exp.zm, linestyle=linestyles[2], color=exp.color) 
#              , label=r"$\frac{\partial \tau}{\partial z}$")
    # turbulent flux
    plt.plot(dturb, exp.zm, 
             label= r"$-\frac{\partial}{\partial z} \langle \overline{w'u'} \rangle$",
             linewidth=1, linestyle=":", color='xkcd:light brown')
    # dispersive flux
    plt.plot(ddisp, exp.zm, 
             label=r"$-\frac{\partial}{\partial z} \langle \bar{w}''\bar{u}'' \rangle$", 
             linewidth=1, linestyle="--", color='xkcd:greyish purple')
    # SGS flux
    plt.plot(dvisc, exp.zm, 
             label=r'$\frac{\partial}{\partial z} \nu_{SGS} \frac{\partial u}{\partial z}$',  
             linestyle="-.", linewidth=0.6, color='xkcd:grey green')
    
    # forcing label
    plt.annotate("$F$", xy=(exp.forcing[-1], exp.zm[int(exp.zsize/2)]), xycoords='data', 
                 xytext=(10, 10), textcoords='offset points', color=exp.color)
    # drag label
    plt.annotate("$-D$", xy=(exp.dragforce[int(exp.zh_index - 1)], exp.zm[int(exp.zh_index)]), xycoords='data', 
                 xytext=(10, 10), textcoords='offset points', color=exp.color)
    # flux label
    plt.annotate(r"$\frac{\partial \tau}{\partial z}$", 
                 xy=(-exp.dragforce[int(exp.zh_index - 1)], exp.zm[int(exp.zh_index)]), xycoords='data', 
                 xytext=(-15, 10), textcoords='offset points', color=exp.color)

    # z_max
    plt.axhline(y=exp.zmax, linewidth=0.5, color='grey')
    if not exp.zmax == exp.zh:
        plt.annotate(r"$z_{\max}$", xy=(0, exp.zmax/exp.zsize), xycoords='axes fraction', 
                     xytext=(-21, 10), textcoords='offset points', color='grey')
    # z_h
    plt.axhline(y=exp.zh, linewidth=0.5,  color="grey")
    plt.annotate(r"$z_H$", xy=(0, exp.zh/exp.zsize), xycoords='axes fraction', 
                 xytext=(-21, 0), textcoords='offset points', color='grey')


    plt.legend(fontsize='small', labelspacing=0.5)
    plt.xlabel(r"Forces and transport [m/s$^2$]")
    plt.ylabel('Height z [m]')
    plt.title("Forces")


    # fluxes
    ax2 = fig.add_subplot(1, 2, 2)
    # stress
    plt.plot(exp.totstress, exp.zm, linestyle=linestyles[0], color=exp.color)
    # drag
    plt.plot(exp.drag, exp.zm, linestyle=linestyles[11], color=exp.color) 
#              ,label=r"$\hat D$")
    # total flux
    plt.plot(exp.totflux, exp.zm, linestyle=linestyles[2], color=exp.color)
#              , label=r"$\tau$")
    # turbulent flux
    plt.plot(exp.turbflux, exp.zm, 
             label=r"$-\langle \overline{w'u'} \rangle$", 
             linewidth=1, linestyle=":", color='xkcd:light brown')
    
    # forcing label
    plt.annotate("$\hat F$", xy=(exp.totstress[3], exp.zm[int(exp.zmax_index/2)+3]), xycoords='data', 
                 xytext=(0, 0), textcoords='offset points', color=exp.color)
    # drag label
    plt.annotate("$\hat D$", xy=(exp.drag[int(exp.zh_index + 1)], exp.zm[int(exp.zh_index + 5)]), xycoords='data', 
                 xytext=(0, 0), textcoords='offset points', color=exp.color)
    # flux label
    plt.annotate(r"$\tau$", 
                 xy=(exp.turbflux[50], exp.zm[-1]/3), xycoords='data', 
                 xytext=(0, 0), textcoords='offset points', color=exp.color)

    # dispersive flux
    plt.plot(exp.dispflux, exp.zm, 
             label=r"$-\langle \bar{w}''\bar{u}'' \rangle$", 
             linewidth=1, linestyle="--", color='xkcd:greyish purple')
    # SGS flux
    plt.plot(exp.viscflux, exp.zm, 
             label=r'$\nu_{SGS} \frac{\partial \langle \bar{u} \rangle}{\partial z}$', 
             linestyle="-.", linewidth=0.6, color='xkcd:grey green')
    
    # z_max
    plt.axhline(y=exp.zmax, linewidth=0.5, color='grey')
    if not exp.zmax == exp.zh:
        plt.annotate(r"$z_{\max}$", xy=(0, exp.zmax/exp.zsize), xycoords='axes fraction', 
                 xytext=(-21, 10), textcoords='offset points', color='grey')
    # z_h
    plt.axhline(y=exp.zh, linewidth=0.5,  color="grey")
    plt.annotate(r"$z_H$", xy=(0, exp.zh/exp.zsize), xycoords='axes fraction', 
                 xytext=(-21, 0), textcoords='offset points', color='grey')

    plt.legend(fontsize='small', labelspacing=0.5)
    plt.xlabel(r"Fluxes, drag, stress [m$^2$/s$^2$]")
    plt.ylabel('Height z [m]')
    plt.title("Integrated forces")
    
    plt.suptitle("%s" % exp.reference )
    plt.tight_layout()
    plt.subplots_adjust(top=0.88)
    plt.show()    
    

[Back to Contents](#Contents)

## Force balance with ISL and RSL

In [ ]:
for exp in exps:
    # plot figure
    fig = plt.figure(figsize=(4*2, 3*2))
    textcol = plt.xlabel('').get_color()  # get default text color
    normh = exp.normh
    normf = exp.normf*exp.zsize
    # define fluxes (old data doesn't have the gradients)    
    dtau = exp.totflux_gradient
    dturb =  exp.turbflux_gradient
    ddisp =  exp.dispflux_gradient
    dvisc =  exp.viscflux_gradient
    
    # forcing    
    plt.plot(exp.forcing*normf, exp.zm*normh, 
             linestyle=linestyles[0], color=exp.color,
             label=r"$F$")
    # drag
    plt.plot(exp.dragforce*normf, exp.zm*normh, 
             linestyle=linestyles[11], color=exp.color,
             label=r"$-D$")
    # total flux
    plt.plot(dtau*normf, exp.zm*normh, 
             linestyle=linestyles[2], color=exp.color,
             label=r"$\frac{\partial \tau}{\partial z}$")
    # turbulent flux
    plt.plot(dturb*normf, exp.zm*normh, 
             linewidth=1, linestyle=":", color='xkcd:light brown',
             label= r"$-\frac{\partial}{\partial z} \langle \overline{w'u'} \rangle$")
    # dispersive flux
    plt.plot(ddisp*normf, exp.zm*normh, 
             linewidth=1, linestyle="--", color='xkcd:greyish purple',
             label=r"$-\frac{\partial}{\partial z} \langle \bar{w}''\bar{u}'' \rangle$")
    # SGS flux
    plt.plot(dvisc*normf, exp.zm*normh, 
             linestyle="-.", linewidth=0.6, color='xkcd:grey green',
             label=r'$\frac{\partial}{\partial z} \nu_{SGS} \frac{\partial u}{\partial z}$')
    # z_h
    if not exp.zmax == exp.zh:
        plt.axhline(y=exp.zh*normh, linewidth=0.5, linestyle=linestyles[2], color=textcol)
        plt.annotate(r"$z_H$", xy=(0, exp.zh/exp.zsize), xycoords='axes fraction', 
                     xytext=(-15, 8), textcoords='offset points', color=textcol)
    # z_RSL
    plt.axhline(y=exp.zrsl*normh, linewidth=0.5, linestyle=linestyles[1], color=textcol)
    # z_ISL
    plt.axhline(y=exp.zisl*normh, linewidth=0.5, linestyle=linestyles[1], color=textcol)
    # layers ISL, RSL, UCL
    plt.annotate("ISL", xy=(0.9, 0.55), xycoords='axes fraction')
    plt.annotate("RSL", xy=(0.9, 0.3), xycoords='axes fraction')
    plt.annotate("UCL", xy=(0.9, 0.05), xycoords='axes fraction')

    plt.legend(labelspacing=0.5)
    plt.xlabel(r"Normalised forces and transport")
    plt.ylabel('Height $z/z_H$')
    plt.title("Forces")

    plt.tight_layout()
    plt.show()
    
    print(exp.reference, "z_ISL =", exp.zisl, "m,",
          "z_ISL/z_H =", round(exp.zisl/exp.zh, 2),
          ", z_ISL/z_max =", round(exp.zisl/exp.zmax, 2))


[Back to Contents](#Contents)
## Stress profiles (CSA)

In [ ]:
# setup
if len(exps) == 7:
    n = len(exps) - 1  # ignore reference solution
    col = 3
elif len(exps) == 1:
    n = 1
    col = 1
else:
    n = len(exps)
    col = 3
row = math.ceil(n/col)
fig = plt.figure(figsize=(3*col*2, 4*row*2))
i = 0
for exp in exps[:n]:
    i += 1
    ax = fig.add_subplot(row, col, i)
#     fig = plt.figure(figsize=(3, 4))  # extra figure for each exp
    # stress
    plt.plot(exp.totstress, exp.zm, color=exp.color, linestyle=":",
            label=r"$\hat F$")
    # drag
    plt.plot(exp.drag, exp.zm, linestyle="-.", color=exp.color, 
             label=r"$\hat D$")
    # total flux
    plt.plot(exp.totflux, exp.zm, color=exp.color, label=r"$\tau$")
    # turbulent flux
    plt.plot(exp.turbflux, exp.zm, 
             label=r"$\langle \overline{w'u'} \rangle$", 
             linewidth=1, linestyle=":", color='xkcd:light brown')
    # dispersive flux
    plt.plot(exp.dispflux, exp.zm, 
             label=r"$\langle \bar{w}''\bar{u}'' \rangle$", 
             linewidth=1, linestyle="--", color='xkcd:greyish purple')
#     # SGS flux
#     plt.plot(exp.viscflux, exp.zm, 
#              label=r'$\nu_{SGS} \frac{\partial \langle \bar{u} \rangle}{\partial z}$', 
#              linestyle="-.", linewidth=0.6, color='xkcd:grey green')
#     # z_max
#     plt.axhline(y=exp.zmax, linewidth=0.5, color='grey')
#     if not exp.zmax == exp.zh:
#         plt.annotate(r"$z_{\max}$", xy=(0, exp.zmax/exp.zsize), xycoords='axes fraction', 
#                  xytext=(-21, 10), textcoords='offset points', color='grey')
#     # z_h
#     plt.axhline(y=exp.zh, linewidth=0.5,  color="grey")
#     plt.annotate(r"$z_H$", xy=(0, exp.zh/exp.zsize), xycoords='axes fraction', 
#                  xytext=(-21, 0), textcoords='offset points', color='grey')
    
    plt.legend(fontsize='small', labelspacing=0.5)
    plt.xlabel(r"Fluxes, drag, stress $Q$ [m$^2$/s$^2$]")
    plt.ylabel('Height z [m]')
    plt.title(exp.reference)
    
plt.suptitle("Stress profiles (CSA)")
plt.tight_layout()
plt.subplots_adjust(top=0.92)
plt.show()

#### Normalised

In [ ]:
# setup
if len(exps) == 7:
    n = len(exps) - 1  # ignore reference solution
    col = 3
elif len(exps) == 1:
    n = 1
    col = 1
else:
    n = len(exps)
    col = 3
row = math.ceil(n/col)
fig = plt.figure(figsize=(3*col*2, 4*row*2))
i = 0
for exp in exps[:n]:
    i += 1
    ax = fig.add_subplot(row, col, i)
#     fig = plt.figure(figsize=(3, 4))  # extra figure for each exp
    # stress
    plt.plot(exp.totstress*exp.normf, exp.zm*exp.normh, color=exp.color, linestyle=":",
            label=r"$\hat F/u_{\ast}^2$")
    # drag
    plt.plot(exp.drag*exp.normf, exp.zm*exp.normh, linestyle="-.", color=exp.color, 
             label=r"$\hat D/u_{\ast}^2$")
    # total flux
    plt.plot(exp.totflux*exp.normf, exp.zm*exp.normh, color=exp.color, label=r"$\tau/u_{\ast}^2$")
    # turbulent flux
    plt.plot(exp.turbflux*exp.normf, exp.zm*exp.normh, 
             label=r"$\langle \overline{w'u'} \rangle/u_{\ast}^2$", 
             linewidth=1, linestyle=":", color='xkcd:light brown')
    # dispersive flux
    plt.plot(exp.dispflux*exp.normf, exp.zm*exp.normh, 
             label=r"$\langle \bar{w}''\bar{u}'' \rangle/u_{\ast}^2$", 
             linewidth=1, linestyle="--", color='xkcd:greyish purple')
#     # SGS flux
#     plt.plot(exp.viscflux*exp.normf, exp.zm*exp.normh, 
#              label=r'$\nu_{SGS} \frac{\partial \langle \bar{u} \rangle}{\partial z}$', 
#              linestyle="-.", linewidth=0.6, color='xkcd:grey green')
    # z_max
    plt.axhline(y=exp.zmax*exp.normh, linewidth=0.5, color='grey')
    if not exp.zmax == exp.zh:
        plt.annotate(r"$z_{\max}$", xy=(0, exp.zmax/exp.zsize), xycoords='axes fraction', 
                 xytext=(-35, 5), textcoords='offset points', color='grey')
    # z_h
    plt.axhline(y=exp.zh*exp.normh, linewidth=0.5,  color="grey")
    plt.annotate(r"$z_H$", xy=(0, exp.zh/exp.zsize), xycoords='axes fraction', 
                 xytext=(-25, 5), textcoords='offset points', color='grey')
    
    plt.legend(fontsize='small', labelspacing=0.5)
    plt.xlabel(r"Normalised stresses")
    plt.ylabel('Height $z/z_H$')
    plt.title(exp.reference)
    
plt.suptitle("Normalised stress profiles")
plt.tight_layout()
plt.subplots_adjust(top=0.92)
plt.show()

[Back to Contents](#Contents)
## Force balance example

In [ ]:
# define which experiment
exp = da.tools.getexp(exps, '(a)')

fig = plt.figure(figsize=(8, 4))
textcol = plt.xlabel('').get_color()  # get default text color
if textcol is 'k':
    textcol2 = 'xkcd:dark grey'
    textcol3 = 'xkcd:slate'
else:  # for dark background
    textcol2 = 'xkcd:light grey'
    textcol3 = 'xkcd:grey'

normh = 1/exp.zmax
normf = exp.csa*exp.normf*exp.zsize

# forcing    
plt.plot(exp.forcing*normf, exp.zm*normh, 
         linewidth=1, color=emphcolors[3])
# drag
plt.plot(exp.dragforce*normf, exp.zm*normh, 
         linewidth=1, color=emphcolors[1])
# total flux
plt.plot(exp.totflux_gradient*normf, exp.zm*normh, 
         linewidth=1, color=emphcolors[0])
# turbulent flux
plt.plot(exp.turbflux_gradient*normf, exp.zm*normh,
         linewidth=1, linestyle=linestyles[9], color=textcol3)
# dispersive flux
plt.plot(exp.dispflux_gradient*normf, exp.zm*normh, 
         linewidth=1, linestyle=linestyles[10], color=textcol3)

# fill between to indicate force balance
plt.fill_betweenx(exp.zm*normh, 0, exp.forcing*normf,
                  color=emphcolors[3], alpha=0.5)
plt.fill_betweenx(exp.zm*normh, 0, exp.dragforce*normf,
                  color=emphcolors[1], alpha=0.5)
plt.fill_betweenx(exp.zm*normh, 0, exp.totflux_gradient*normf,
                  color=emphcolors[0], alpha=0.4)

# text annotations
plt.annotate('momentum source', 
             xy=(0.55, 0.78), xycoords='axes fraction', 
             rotation=270, fontsize=9, color=emphcolors[3])
plt.annotate('momentum transport', 
             xy=(0.42, 0.78), xycoords='axes fraction', 
             rotation=270, fontsize=9, color=emphcolors[0])
plt.annotate('momentum sink', 
             xy=(0.22, 0.25), xycoords='axes fraction', 
             fontsize=9, color=emphcolors[1])
plt.annotate('momentum transport', 
             xy=(0.59, 0.25), xycoords='axes fraction', 
             fontsize=9, color=emphcolors[0])

# forcing label
plt.annotate("$F$", 
             xy=(0.55, 0.28), xycoords='axes fraction', 
             color=emphcolors[3])
# drag label
plt.annotate("$-D$", 
             xy=(0.34, 0.1), xycoords='axes fraction', 
             color=emphcolors[1])
# flux label
plt.annotate(r"$\frac{\partial \tau}{\partial z}$", 
             xy=(0.62, 0.1), xycoords='axes fraction', 
             color=emphcolors[0])
# turb flux label
plt.annotate(r"$-\frac{\partial}{\partial z} \langle \overline{w'u'} \rangle$", 
             xy=(0.35, 0.88), xycoords='axes fraction',
             color=textcol2)
# disp flux label
plt.annotate(r"$-\frac{\partial}{\partial z} \langle \bar{w}''\bar{u}'' \rangle$", 
             xy=(0.52, 0.88), xycoords='axes fraction', 
             color=textcol2)

# z_UCL
plt.axhline(y=exp.zucl*normh, linewidth=0.5, linestyle=linestyles[1],
            color='grey')
# z_RSL
plt.axhline(y=exp.zrsl*normh, linewidth=0.5, linestyle=linestyles[1],
            color='grey')

# layers ISL, RSL, UCL
plt.annotate("ISL", xy=(0.85, 0.6), xycoords='axes fraction')
plt.annotate("RSL", xy=(0.85, 0.28), xycoords='axes fraction')
plt.annotate("UCL", xy=(0.85, 0.08), xycoords='axes fraction')

plt.xlabel(r"Normalised forces")
plt.ylabel('Height $z/z_H$')
plt.title("Force balance")
maximum = np.max(exp.turbflux_gradient*normf)
lims = maximum*0.96
plt.xlim([-lims, lims])
plt.show()

[Back to Contents](#Contents)
## Difference CSA and ISA

In [ ]:
# define which experiment
exp = da.tools.getexp(exps, '(a)')

fig = plt.figure(figsize=(3*2, 4*1))
# CSA
ax1 = fig.add_subplot(1, 2, 1)

textcol = plt.xlabel('').get_color()  # get default text color
if textcol is 'k':
    textcol2 = 'xkcd:dark grey'
    textcol3 = 'xkcd:slate'
else:  # for dark background
    textcol2 = 'xkcd:light grey'
    textcol3 = 'xkcd:grey'

normh = 1/exp.zmax
normf = exp.normf

# forcing    
plt.plot(exp.totstress*normf, exp.zm*normh, 
         linewidth=1, color=emphcolors[3])
# drag
plt.plot(exp.drag*normf, exp.zm*normh, 
         linewidth=1, color=emphcolors[1])
# total flux
plt.plot(exp.totflux*normf, exp.zm*normh, 
         linewidth=1, color=emphcolors[0])
# turbulent flux
plt.plot(exp.turbflux*normf, exp.zm*normh,
         linewidth=1, linestyle=linestyles[9], color=textcol3)
# dispersive flux
plt.plot(exp.dispflux*normf, exp.zm*normh, 
         linewidth=1, linestyle=linestyles[10], color=textcol3)

# forcing label
plt.annotate("$\hat F$", 
             xy=(0.92, 0.13), xycoords='axes fraction', 
             color=emphcolors[3])
# drag label
plt.annotate("$\hat D$", 
             xy=(0.6, 0.08), xycoords='axes fraction', 
             color=emphcolors[1])
# flux label
plt.annotate(r"$\tau$", 
             xy=(0.25, 0.08), xycoords='axes fraction', 
             color=emphcolors[0])
# turb flux label
plt.annotate(r"$- \langle \overline{u'w'} \rangle$", 
             xy=(0.38, 0.35), xycoords='axes fraction',
             color=textcol2)
# disp flux label
plt.annotate(r"$- \langle \bar{u}''\bar{w}'' \rangle$", 
             xy=(0.08, 0.45), xycoords='axes fraction', 
             color=textcol2)

# z_UCL
plt.axhline(y=exp.zucl*normh, linewidth=0.5, linestyle=linestyles[1],
            color='grey')

plt.xlabel("Normalised stresses")
plt.ylabel('Height $z/z_H$')
plt.title("CSA")

# ISA
ax1 = fig.add_subplot(1, 2, 2)

normf = exp.isa*exp.normf  # change to ISA

# forcing    
plt.plot(exp.totstress*normf, exp.zm*normh, 
         linewidth=1, color=emphcolors[3])
# drag
plt.plot(exp.drag*normf, exp.zm*normh, 
         linewidth=1, color=emphcolors[1])
# total flux
plt.plot(exp.totflux*normf, exp.zm*normh, 
         linewidth=1, color=emphcolors[0])
# turbulent flux
plt.plot(exp.turbflux*normf, exp.zm*normh,
         linewidth=1, linestyle=linestyles[9], color=textcol3)
# dispersive flux
plt.plot(exp.dispflux*normf, exp.zm*normh, 
         linewidth=1, linestyle=linestyles[10], color=textcol3)

# forcing label
plt.annotate("$\hat F$", 
             xy=(0.92, 0.13), xycoords='axes fraction', 
             color=emphcolors[3])
# drag label
plt.annotate("$\hat D$", 
             xy=(0.6, 0.08), xycoords='axes fraction', 
             color=emphcolors[1])
# flux label
plt.annotate(r"$\tau$", 
             xy=(0.25, 0.08), xycoords='axes fraction', 
             color=emphcolors[0])
# turb flux label
plt.annotate(r"$- \langle \overline{u'w'} \rangle$", 
             xy=(0.38, 0.35), xycoords='axes fraction',
             color=textcol2)
# disp flux label
plt.annotate(r"$- \langle \bar{u}''\bar{w}'' \rangle$", 
             xy=(0.08, 0.45), xycoords='axes fraction', 
             color=textcol2)

# z_UCL
plt.axhline(y=exp.zucl*normh, linewidth=0.5, linestyle=linestyles[1],
            color='grey')

plt.xlabel("Normalised stresses")
plt.ylabel('Height $z/z_H$')
plt.title("ISA")

plt.tight_layout()
plt.show()

[Back to Contents](#Contents)

---
# Parameter estimations

## Friction velocity estimates

In [ ]:
# setup
if len(exps) == 1:
    n = 1
    col = 1
else:
    n = len(exps)
    col = 2
row = math.ceil(n/col)
i = 0
fig = plt.figure(figsize=(3*col, 4*row))
for exp in exps[:n]:
    i += 1
    ax = fig.add_subplot(row, col, i)
#     fig = plt.figure(figsize=(3, 4))  # extra figure for each exp
    # totalflux
    plt.plot(exp.totflux, exp.zt, color=exp.color, label="total flux")
    # totalflux line interpolation
    plt.plot(exp.totflux_interpolated, exp.zt, color="grey", linestyle="--", label="interpolated flux")
    # ustar^2 max
    ustar_max_index = np.argmax(abs(exp.totflux))
    plt.scatter(exp.ustar_max**2, exp.zt[ustar_max_index], marker='.', color='k')
    plt.text(exp.ustar_max**2-0.0005, exp.zt[ustar_max_index]+10, r"$u_{\ast,\max}^{2}$", fontsize=8, color='k')
    # ustar^2 line extrapolation
    plt.scatter(exp.ustar_best**2, exp.zt[0], marker='.', color='k')
    plt.text(exp.ustar_best**2-0.0005, exp.zt[0]+20, r"$u_{\ast,best}^{2}$", fontsize=8, color='k')
    # ustar^2 stress csa/isa
    plt.scatter(exp.ustar**2, exp.zt[0], marker='.', color='k')
    plt.text(exp.ustar**2-0.0005, exp.zt[0]+10, r"$u_{\ast,stress}^{2}$", fontsize=8, color='k')
    # z_h
    plt.axhline(y=exp.zh, linewidth=0.5,  color="grey")
    plt.scatter(0, exp.zh, marker='.', color='grey')
    plt.text(0+0.0002, (exp.zh-8), "$z_H$", fontsize=8, color='grey')
    # z_max
    plt.axhline(y=exp.zmax, linewidth=0.5, color='grey')
    plt.scatter(0, exp.zmax, marker='.', color='grey')
    plt.text(0+0.0002, (exp.zmax+8), r"$z_{\max}$", fontsize=8, color='grey')

    plt.xlabel(r"$Q_i$")
    plt.ylabel(r'$z$')
    plt.legend()
    plt.title(exp.reference)
    plt.tight_layout()
plt.suptitle(r"Estimations for friction velocity $u_{\ast}$")
plt.tight_layout()
plt.subplots_adjust(top=0.94)
plt.show()

for exp in exps:
    print(exp.reference)
    print("u* estimated from total flux maximum:      ", exp.ustar_max)
    print("u* estimated from total flux extrapolation:", exp.ustar_best)

[Back to Contents](#Contents)
## Blending height estimates

In [ ]:
# setup
if len(exps) == 7:
    n = len(exps) - 1  # ignore reference solution
    col = 3
elif len(exps) == 1:
    n = 1
    col = 1
else:
    n = len(exps)
    col = 3
row = math.ceil(n/col)
i = 0
# show blending heights
fig = plt.figure(figsize=(3*col, 4*row))
for exp in exps[:n]:
    i += 1
    ax = fig.add_subplot(row, col, i)
#     fig = plt.figure(figsize=(3, 4))  # extra figure for each exp
    plt.plot(exp.totflux*exp.normf, exp.zm*exp.normh, color=exp.color, label="total flux")
    plt.plot(exp.dispflux*exp.normf, exp.zm*exp.normh, linestyle="--", color=exp.color, label="dispersive flux")
    # z_H
    plt.axhline(y=exp.zh*exp.normh, linewidth=0.5,  color="grey")
    plt.scatter(0, exp.zh*exp.normh, marker='.', color='grey')
    plt.text(0-0.05, (exp.zh-8)*exp.normh, "$z_H$", fontsize=8, color='grey')
    # z_max
    plt.axhline(y=exp.zmax*exp.normh, linewidth=0.5, color='grey')
    plt.scatter(0, exp.zmax*exp.normh, marker='.', color='grey')
    plt.text(0-0.05, (exp.zmax+6)*exp.normh, "$z_{max}$", fontsize=8, color='grey')
    # z_r
    plt.axhline(y=exp.zr_threshold*exp.normh, linewidth=0.6, color='red')
    z1 = plt.scatter(0, exp.zr_threshold*exp.normh, marker='.', color='red')
    plt.axhline(y=exp.zr_local_minimum*exp.normh, linewidth=0.6,  color='green')
    z2 = plt.scatter(0, exp.zr_local_minimum*exp.normh, marker='.', color='green')
    plt.axhline(y=exp.zr_absolute_minimum*exp.normh, linewidth=0.6, color='blue')
    z3 = plt.scatter(0, exp.zr_absolute_minimum*exp.normh, marker='.', color='blue')

    plt.text(0+0.05, (exp.zr_threshold-8)*exp.normh, "$z_{r,1}$", fontsize=8, color='r')
    plt.text(0.7, 0.6, r'$z_{r,1}$: threshold', {'color': 'r'}, transform = ax.transAxes)
    plt.text(0+0.05, (exp.zr_local_minimum-8)*exp.normh, "$z_{r,2}$", fontsize=8, color='g')
    plt.text(0.7, 0.55, r'$z_{r,2}$: local min', {'color': 'g'}, transform = ax.transAxes)
    plt.text(0+0.05, (exp.zr_absolute_minimum-8)*exp.normh, "$z_{r,3}$", fontsize=8, color='b')
    plt.text(0.7, 0.5, r'$z_{r,3}$: abs min', {'color': 'b'}, transform = ax.transAxes)
    
    plt.title(exp.reference)
    plt.xlabel(r"$Q_i/u_*^2$")
    plt.ylabel(r'$z/z_H$')
    plt.legend(loc=1)
    plt.tight_layout()
plt.suptitle("Blending height $z_r$ estimation with normalised momentum fluxes")
plt.tight_layout()
plt.subplots_adjust(top=0.92)
plt.show()

[Back to Contents](#Contents)
## Log law estimates

In [ ]:
# define index for reference height 
tref = 60
for exp in exps:
    # get fit range from fits above
    fitrange = slice(exp.z0_fitstart, exp.z0_fitstop)
    
    # plot figure
    fig = plt.figure(figsize=(4*2, 3*1))
    # mean wind
    ax = fig.add_subplot(1, 2, 1)
    plt.plot(exp.uav, exp.zt*exp.normh, label=r'Wind speed $\langle \bar u\rangle$', color=exp.color)
    # log law
    plt.plot(exp.uav, exp.loglaw*exp.normh, label="log law", color='xkcd:bluish grey', linestyle="--", linewidth=0.6)
    # z_0 + z_d
    plt.scatter(exp.uav[0], exp.loglaw[0]*exp.normh, marker='.', color='k')
    plt.text(exp.uav[0], (exp.loglaw[0]+10)*exp.normh, "$z_0 + z_d$", fontsize=8)   
    # z_H
    plt.scatter(exp.uav[exp.zh_index], exp.zh*exp.normh, marker='.', color='grey')
    plt.text(exp.uav[exp.zh_index]-0.05, (exp.zh-10)*exp.normh, "$z_H$", fontsize=8, color='grey')
    # z_max
    plt.scatter(exp.uav[exp.zmax_index], exp.zmax*exp.normh, marker='.', color='grey')
    plt.text(exp.uav[exp.zmax_index], (exp.zmax-10)*exp.normh, "$z_{max}$", fontsize=8, color='grey')
    # z_e
    try:
        plt.scatter(exp.uav[exp.ze_index], exp.ze*exp.normh, marker='.', color='k')
        plt.text(exp.uav[exp.ze_index], (exp.ze+10)*exp.normh, "$z_e$", fontsize=8, color='k')
    except Exception:
        pass
    # z_r used for fitting
    plt.scatter(exp.uav[exp.zr_index], exp.zr*exp.normh, marker='.', color='r')
    plt.text(exp.uav[exp.zr_index], (exp.zr+10)*exp.normh, "$z_r$", fontsize=8, color='r')

    plt.legend()
    plt.xlabel(r'Wind speed $\langle \bar u\rangle$')
    plt.ylabel('Height $z/z_H$')
    plt.title("Mean wind velocity %s"  % exp.reference)

    # law of the wall
    ax = fig.add_subplot(1, 2, 2)

    # define semilog axis that maskes any nan entry in log
    logz = np.ma.log(exp.zt - exp.zd)  

    plt.plot(logz, exp.uav, label=r'Wind speed $\langle \bar u\rangle$', color=exp.color)
    plt.plot(logz[fitrange], exp.uav[fitrange], label="log law fit", color='xkcd:bluish grey', linestyle="--", linewidth=0.6)
    # fitrange
    plt.scatter(logz[exp.z0_fitstart], exp.uav[exp.z0_fitstart], marker='.', color='grey')
    plt.scatter(logz[exp.z0_fitstop], exp.uav[exp.z0_fitstop], marker='.', color='grey')
    # z_r
    if np.logical_not(np.isnan(logz[exp.zr_index])):
        plt.scatter(logz[exp.zr_index], exp.uav[exp.zr_index], marker='.', color='r')
        plt.text(logz[exp.zr_index]+0.2, exp.uav[exp.zr_index], "$z_r$", fontsize=8, color='r')

    plt.legend()
    plt.xlabel('Logarithmic height $\log(z-z_d)$')
    plt.ylabel(r'Wind speed $\langle \bar u\rangle$')
    plt.title("Law of the wall %s"  % exp.reference)

    plt.tight_layout()
    plt.show()

    print(exp.reference)
    print("velocity at boundary (z=", exp.zt[0], "m):  ", round(exp.uav[0], 3), "m/s")
    print("reference velocity (z=", exp.zt[tref], "m):   ", round(exp.uav[tref], 3), "m/s")
    print("free stream velocity (z=", exp.zt[-10], "m):", round(exp.uav[-10], 3), "m/s\n")
    print("z_r: ", exp.zr, "m")
    print("z_H:", exp.zh, 'm \n')
    print("Log law fits between", exp.zt[exp.z0_fitstart], "m and", exp.zt[exp.z0_fitstop], "m.")
    print("displacement length z_d = ", exp.zd, "    with correlation: ", exp.zd_correlation)
    print("roughness length    z_0 = ", exp.z0)
    print("friction velocity    u* = ", exp.ustar_loglaw)
    print("friction velocity from fluxes u* = ", round(exp.ustar, 3))
    print("z_d/z_H = ", round(exp.zd*exp.normh, 3))
    print("z_0/z_H = ", round(exp.z0*exp.normh, 3))
    print("(z_0 + z_d)/z_H = ", round((exp.zd+exp.z0)*exp.normh, 3))
    print("u*/u_ref = ", round(exp.ustar/exp.uav[tref], 3))

[Back to Contents](#Contents)

---
# Field plots

#### Read in field data

In [ ]:
# add data paths
projectpath, exppath, workpath, ephemeral = da.tools.environment()
for exp in exps:
#     datapath = os.path.join(ephemeral, exp.name, '')
    datapath = os.path.join(workpath, exp.name, '')
    filelist = os.path.join(datapath, "*dump." + exp.name + "*")
    ncpaths = glob.glob(filelist)
    # get fields
    ncfields = da.tools.getncdata(ncpaths)
    exp.add_data('ncfields', ncfields)
    print("Available variables:\n")
    for fdata in exp.ncfields:
        print(fdata.title, ":\n", list(fdata.variables.keys()))

#### Select variables to read in:

In [ ]:
# dict of dict with selected variables
selectvars = {
             'fielddump' : {'tfield' : 'time',  
                            'u' : 'u', 
                            'v' : 'v', 
                            'w' : 'w',
                            'p' : 'pres' 
                           },
            'tdump' : {'tstats' : 'time', 
                       'ut' : 'ut', 'vt' : 'vt', 'wt' : 'wt',
                       'upwpt' : 'upwpt', 'vpwpt' : 'vpwpt', 
                       'tket' : 'tketc',
                       'upupt' : 'upuptc', 'vpvpt' : 'vpvptc', 'wpwpt' : 'wpwptc',
                       'pt' : 'pt'
                      }
            }
for exp in exps:
    exp = da.tools.getncvars(exp, selectvars)

#### Print available fields

In [ ]:
for exp in exps:
    # all available variables
    ncvarnames = []  # list with names of variables in exp.data
    for name, variable in exp.data.items():
        if isinstance(variable, (netCDF4._netCDF4.Variable)):
            ncvarnames.append(name)    
            
    # print only 3D field variables:
    fieldaxes = ['xt', 'xm', 'yt', 'ym']
    ncfields3D = []
    ncfields3Dnames = []
    for name in ncvarnames:
        variable = exp.data[name]
        if any(axis in fieldaxes for axis in variable.dimensions):
            if not variable.name in fieldaxes:  # do not print axis variables themselves
                ncfields3D.append(variable)
                ncfields3Dnames.append(name)
    print(exp.name, "Available 2D or 3D fields:\n", sorted(ncfields3Dnames), "\n")
#     print(exp.name, "Available 2D or 3D fields:\n", ncfields3D, "\n")

[Back to Contents](#Contents)
## Field plot

In [ ]:
# define field variable
varname = 'ut'
# define positions for field plot
tpos = -1
spos = 10
# define fixed axis for field plot
fix = 'zaxis'
# define plot limits
# limits=[0, 100, 0, 100]
limits=None
# define a colour map
cbarmap = plt.cm.get_cmap('RdYlBu_r')

for exp in exps:    
    field = da.plot.prepare_fielddata(exp, varname, 
                                      fix, spos, tpos, 
                                      limits)
            
    field = da.plot.prepare_contourplot(field, cbarmap, 
                                        cbartype='centered')
    
    da.plot.contourfield(field)

[Back to Contents](#Contents)
## Field plot with mini layout

In [ ]:
# define field variable
varname = 'ut'
# define positions for field plot
tpos = -1
spos = 10
# define fixed axis for field plot
fix = 'zaxis'
# define plot limits
limits=None

for exp in exps:    
    field = da.plot.prepare_fielddata(exp, varname, 
                                      fix, spos, tpos, 
                                      limits)
              
    field = da.plot.prepare_contourplot(field)
    
    field = da.plot.prepare_layoutplot(field, divx=4, divy=2)

    da.plot.layout_fieldplot(field)

[Back to Contents](#Contents)
## Quiver and wind speed plot

In [ ]:
# define wind variables
var1 = 'ut'
var2 = 'vt'
var3 = 'wt'
# define positions for field plot
tpos = -1
spos = 10
# define fixed axis for field plot
fix = 'zaxis'
# define plot limits
#     limits=[0, 100, 0, 100]
limits=None
sparse=3
# define colour map
# cbarmap = plt.cm.get_cmap('YlOrRd')
cbarmap = plt.cm.get_cmap('RdYlBu_r')

for exp in exps:    
    field = da.plot.prepare_quiverplot(exp, var1, var2, var3, fix,
                               spos, tpos, limits, sparse)
    
    field = da.plot.prepare_contourplot(field, cbarmap)  
    
    field = da.plot.prepare_layoutplot(field)

    da.plot.layout_fieldplot(field, quiver=True)

[Back to Contents](#Contents)
## Field plot with transformed field

In [ ]:
# define field variable
varname = 'ut'
# define positions for field plot
tpos = -1
spos = 10
# define fixed axis for field plot
fix = 'yaxis'
# define plot limits
# limits=[0, 100, 0, 100]
limits=None

for exp in exps:    
    field = da.plot.prepare_fielddata(exp, varname, 
                                      fix, spos, tpos, 
                                      limits)
    
    # transform field
    # ut - <ut> instead
    tmpfield = field.data.copy()
    field.data = (tmpfield.transpose() - exp.uav).transpose()
    field.name =  r'$\bar{u} - \langle \bar{u} \rangle$'
        
    field = da.plot.prepare_contourplot(field)

    da.plot.contourfield(field)

[Back to Contents](#Contents)
## Compare field plots with equal limits

In [ ]:
# define field variable
varname = 'ut'
# define positions for field plot
tpos = -1
spos = 10
# define fixed axis for field plot
fix = 'zaxis'
# define plot limits
# limits=[0, 100, 0, 100]
limits=None
# define a colour map
cbarmap = plt.cm.get_cmap('RdYlBu_r')

fields = []
for exp in exps:   
    field = da.plot.prepare_fielddata(exp, varname, 
                                      fix, spos, tpos, 
                                      limits)
    fields.append(field)
fields = da.plot.fieldcontours(fields, cbarmap, 
                               cbartype='centered')

for field in fields:
    field.block_opts = {'facecolor': 'w'}
    plt.figure()
    da.plot.contourfield(field)

[Back to Contents](#Contents)
## Video of field plots

In [ ]:
# define field variable
varname = 'ut'
# define positions for field plot
tpos = slice(0, None)
spos = 10
# define fixed axis for field plot
fix = 'yaxis'
# define plot limits
limits=None

for exp in exps:    
    field = da.plot.prepare_fielddata(exp, varname, 
                                      fix, spos, tpos, 
                                      limits)
            
    field = da.plot.prepare_contourplot(field)
    
    field = da.plot.prepare_layoutplot(field)

    da.plot.layout_fieldplot_video(field, show=True, save=False)
    

[Back to Contents](#Contents)
## Flow field decomposition into time-space average, dispersive and turbulent term.

In [ ]:
for exp in exps:
    # Example data from expnr = '462' with varstart = -56, varstop = -26 and tpos=-8. data: name="decomposition-462"

    # load data from exp object
    dimblocks = exp.dimblocks
    xax = exp.xt
    yax = exp.yt
    # try loading stored fields. if not available, compute them
    try:
        varfield = exp.instantaneous_field
        doubleav = exp.space_time_average
        disfield = exp.dispersive_field
        turbfield = exp.turbulent_field
        print("Loaded pre-computed fields")

    except Exception:
        fielddata = exp.u
        tpos = exp.t_inst_index
        varfield = fielddata[tpos, :, :]
        avfield = np.mean(fielddata, axis=0)
        doubleav = np.mean(np.mean(avfield))
        disfield = avfield - doubleav
        turbfield = varfield - avfield
        print("Calculated fields.")

    print("Space-time average:", doubleav)

    field = varfield
    vmax = np.nanmax(field)
    vmin = np.nanmin(field)
    cmax = math.ceil(vmax*10)/10
    cmin = math.floor(vmin*10)/10
    tmax = math.floor(vmax*10)/10
    tmin = math.ceil(vmin*10)/10
    tex = max(tmax, abs(tmin))
    cex = max(cmax, abs(cmin))
    ticks = np.linspace(-tex, tex, 9)
    levels = np.linspace(-cex, cex, 256)
    contour_opts = {'levels': levels, 'cmap': plt.cm.get_cmap('RdYlBu_r')}

    fig = plt.figure(figsize=plt.figaspect(0.3), dpi=150)

    ax = fig.add_subplot(1, 3, 1)
    cs = plt.contourf(xax[:], yax[:], field, **contour_opts)
    da.plot.layout(dimblocks, ax=ax, facecolor='white')
    plt.axis('scaled')
    plt.xlabel('x (m)')
    plt.ylabel('y (m)')
    plt.title("West-East velocity u")

    ax = fig.add_subplot(1, 3, 2)
    field = disfield
    cs = plt.contourf(xax[:], yax[:], field, **contour_opts)
    da.plot.layout(dimblocks, ax=ax, facecolor='white')
    plt.axis('scaled')
    plt.xlabel('x (m)')
    plt.ylabel('y (m)')
    plt.title("Spatial variations of time mean")

    ax = fig.add_subplot(1, 3, 3)
    field = turbfield
    cs = plt.contourf(xax[:], yax[:], field, **contour_opts)
    da.plot.layout(dimblocks, ax=ax, facecolor='white')
    plt.axis('scaled')
    plt.xlabel('x (m)')
    plt.ylabel('y (m)')
    plt.title("Turbulent fluctuations")

    plt.tight_layout()

    fig.subplots_adjust(right=0.8)
    # The dimensions [left, bottom, width, height] of the new axes. All quantities are in fractions of figure width and heigh
    cbar_ax = fig.add_axes([0.82, 0.14, 0.01, 0.77])
    cbar = fig.colorbar(cs, cax=cbar_ax)
    cbar.set_ticks(ticks)
    cbar.set_label('m/s')

[Back to Contents](#Contents)